In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

In [23]:
dictionary = build_dictionary("words_250000_train.txt")

In [24]:
#find longest word in dictionary
max_len = 0
w = ""
for word in dictionary: 
    if len(word) > max_len:
        max_len = len(word)
        w = word
print(max_len)

29


In [25]:
sets = [set(x) for x in dictionary]

In [26]:
#create unique combinations of letters from a set

import itertools

def get_combinations(letters):
    combinations = []
    for i in range(1,len(letters)+1):
        combinations += list(itertools.combinations(letters, i))
    return combinations

In [27]:
nos = [len(x) for x in sets]

In [28]:
#get frequency of each length
from collections import Counter
freq = Counter(nos)

In [29]:
summ = 0
ss = "abcdefghijklmnopqrstuvwxyz"
for i in freq:
    summ += freq[i]*len(get_combinations(ss[:i]))

In [30]:
summ

100725482

In [32]:
value = {"a":1,"b":2,"c":3,"d":4,"e":5,"f":6,"g":7,
         "h":8,"i":9,"j":10,"k":11,"l":12,"m":13,"n":14,
         "o":15,"p":16,"q":17,"r":18,"s":19,"t":20,"u":21,
         "v":22,"w":23,"x":24,"y":25,"z":26}

In [33]:
def dataframemaker(dicc):    
    dfs = []

    for word in dicc:
        for comb in get_combinations("".join(set(word))):
            #create an array of -1s of length 80
            num = np.full(80,-1)
            guess = np.full(26,0)
            #fill the array with the index of the letters in the word
            k=len(word)
            for i in range(k):
                if(word[i] in comb):
                    num[i] = value[word[i]]
                    num[80-k+i] = value[word[i]]
                else:
                    num[i] = 0
                    num[80-k+i] = 0
                    guess[value[word[i]]-1] = 1
            dfs.append(num.tolist()+guess.tolist())
    
    return pd.DataFrame(dfs,dtype=np.int8)
        

In [ ]:
import multiprocessing as mp

dataset = []

for dicc in np.array_split(dictionary,10):
    
    print("1 \n")

    manager = mp.Manager()
    pool = mp.Pool()

    dictsplit = np.array_split(dicc,128)

    datas = pool.map(dataframemaker,dictsplit)
    pool.close()
    pool.join()

    finaldata = pd.concat(datas).reset_index(drop=True)

    del datas

    finaldata.columns = [str(x) for x in range(80)]+[x for x in "abcdefghijklmnopqrstuvwxyz"]
    dataset.append(finaldata)
    del finaldata

1 

1 

1 

1 



In [ ]:
dataset = pd.concat(dataset).reset_index(drop=True)

In [ ]:
dataset.to_parquet("fd.parquet")